In [9]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [10]:
import ipywidgets as widgets
import io
from ipywidgets import interact,Layout
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import clear_output
from random import sample 
from pymove import MoveDataFrame
from pymove import filters
from io import StringIO
from pymove.visualization import folium as f, matplotlib as mpl
from pymove.visualization.folium import plot_trajectories, plot_markers,plot_points
import folium
from copy import copy
from os import listdir
from IPython.display import display, HTML
import keras
from keras.models import load_model
import h5py
import nltk
import os
from sklearn import metrics
%matplotlib inline

In [11]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [12]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 180px
}
</style>

In [14]:
def setup_ui(df):
    
    out = widgets.Output()
    with out:
        display(df)
    return out

In [15]:
def extrinsic_evaluation(model_name,model_dict):
    
    dados = pd.read_csv("Datasets/"+model_dict[model_name]+".csv", index_col=0)

    X = dados.values[:,:-1]
    Y = dados.values[:,-1]

    model = keras.models.load_model("Models/"+model_name+".h5")

    probs = model.predict(X, verbose=0)

  # predição do modelo
    y_ = np.argmax(model.predict(X, verbose=0), axis=1)

  #Acc = metrics.accuracy_score(Y,y_)
    F1 = metrics.f1_score(Y,y_, average='weighted')

    md = model_name[0:2]  
    if (md == 'LB'):
        range_labes = 371
    else:
        range_labes = 368

    AccTop1 = metrics.top_k_accuracy_score(Y,probs, k=1, labels = np.array(range(0,range_labes)))
    AccTop3 = metrics.top_k_accuracy_score(Y,probs, k=3, labels = np.array(range(0,range_labes)))
    AccTop5 = metrics.top_k_accuracy_score(Y,probs, k=5, labels = np.array(range(0,range_labes)))

    return [model_name, AccTop1, AccTop3, AccTop5, F1]  
    

In [16]:
#@title **Demo Application**

np.seterr(divide='ignore', invalid='ignore')

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab([out1, out2, out3, out4], layout=widgets.Layout(width="1300px"))

tab.set_title(0, 'Upload')
tab.set_title(1, 'View Trajectory Data')
tab.set_title(2, 'Intrinsic Evaluation')
tab.set_title(3, 'Extrinsic Evaluation')

display(tab)

with out1:

    def func1(b):

        global data_up_out
        global data_up
        global data_list 
        global data_out
        global text3_out
        global on_change_dataset
        global data_ml_dict_show
        global emb_data_ml_out
        global text_dataset_out
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        with data_up_out:
              
            
            data_up_out.clear_output()
            
            new_values = b.new
            keys = list(new_values.keys())

            for k in keys:
                data = StringIO(str(new_values[k]["content"],'utf-8'))
                df=pd.read_csv(data,index_col=0)
                df.to_csv("Trajectories/"+k.split(".")[0]+".csv")
             
            data_list = [data.split(".")[0] for data in listdir("Trajectories/")]
            
            if("" in data_list):
                data_list.remove("")
            
            #pd.DataFrame({"Model_Data":model_data_list}).to_csv("Model_Data.csv",index=False)
            
            data_up.options= data_list 
            data_up.value= [data_list[0]]

            display(data_up)
            
        with emb_data_ml_out:

            emb_data_ml_out.clear_output()
            
            keys = list(new_values.keys())
            
            for k in keys:
                data_ml_dict_show[k.split(".")[0]] = k.split(".")[0]

            
            emb_data_ml.options = [data_ml_dict_show[dl] for dl in data_list]
            display(emb_data_ml)
            
        with text3_out:

            text3_out.clear_output()
            text3 = widgets.Dropdown(description='',layout=widgets.Layout(width="80px"),options=data_list)
            text3.observe(on_change_dataset, names='value')
            display(text3)
           
            
            
            
    def func2(b):
        
        global emb_ml_up_out
        global emb_ml_up
        global emb_ml_out
        global emb_list
        global emb_ml
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        with emb_ml_up_out:
              
            
            emb_ml_up_out.clear_output()
            
            new_values = b.new
            keys = list(new_values.keys())

            for k in keys:
                
                data = StringIO(str(new_values[k]["content"],'utf-8'))
                df=pd.read_csv(data,index_col=0)
                df.to_csv("Embeddings/"+k.split(".")[0]+".csv")
             
            emb_list = [emb.split(".")[0] for emb in listdir("Embeddings/")]
            
            if("" in emb_list):
                emb_list.remove("")
            
            pd.DataFrame({"Emb":emb_list}).to_csv("Emb.csv",index=False)
            
            emb_ml_up.options = emb_list
            emb_ml_up.value = [emb_list[0]]

            display(emb_ml_up)
            
        with emb_ml_out:

            emb_ml_out.clear_output()
            emb_ml = widgets.Select(options=emb_list + [''],description='Embeddings:',value='',layout=widgets.Layout(width="380px",height="180px"))
            emb_ml.observe(on_change_emb_ml, names = 'value')
            
                
            display(emb_ml)
            
        
            
        with model_choice_out:
            
            Emb_list = list(pd.read_csv("Emb.csv").Emb)
            Emb_df = pd.read_csv("Emb#Data.csv") 
            Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
            model_choice.options = Emb_list 
            model_choice.value = Emb_list[0]
            model_choice_out.clear_output()
            display(model_choice)
        
    
               
    def on_change_datasets(change):

        global dataset_3
        dataset_3 = change.new
        
    def func3(b):
        
        global model_ml_up_out
        global model_ml_up
        global model_ml_out
        global model_list
        global model_ml
        
        with model_ml_up_out:
              
            
            model_ml_up_out.clear_output()
            
            new_values = b.new
            keys = list(new_values.keys())

            for k in keys:
                data = io.BytesIO(new_values[k]["content"])
                model= h5py.File(data,'r')
                model_keras=load_model(model)
                model_keras.save("Models/"+k.split(".")[0]+".h5")
             
            model_list = [model.split(".")[0] for model in listdir("Models/")]
            
            if("" in model_list):
                model_list.remove("")
            
            pd.DataFrame({"Model":model_list}).to_csv("Model.csv",index=False)
            
            model_ml_up.options= model_list 
            model_ml_up.value = [model_list[0]]

            display(model_ml_up)
            
        with model_ml_out:

            model_ml_out.clear_output()
            model_ml = widgets.Select(options=model_list + [''],description='Models:',value='',layout=widgets.Layout(width="380px",height="180px"))
            model_ml.observe(on_change_model_ml, names = 'value')
    
                
            display(model_ml)
            
            
                
            
    def func4(b):
        
        global model_data_up_out
        global model_data_up
        global model_data_list 
        global model_data_ml_out
        global model_data_ml_dict_show
        global on_change_model_data_ml
        
        with model_data_up_out:
              
            
            model_data_up_out.clear_output()
            
            new_values = b.new
            keys = list(new_values.keys())

            for k in keys:
                data = StringIO(str(new_values[k]["content"],'utf-8'))
                df=pd.read_csv(data,index_col=0)
                df.to_csv("Datasets/"+k.split(".")[0]+".csv")
             
            model_data_list = [model_data.split(".")[0] for model_data in listdir("Datasets/")]
            
            if("" in model_data_list):
                model_data_list.remove("")
            
            #pd.DataFrame({"Model_Data":model_data_list}).to_csv("Model_Data.csv",index=False)
            
            model_data_up.options= model_data_list 
            model_data_up.value= [model_data_list[0]]

            display(model_data_up)
            
        with model_data_ml_out:

            model_data_ml_out.clear_output()
            
            keys = list(new_values.keys())
            
            for k in keys:
                model_data_ml_dict_show[k.split(".")[0]] = k.split(".")[0]

            
            model_data_ml.options = [model_data_ml_dict_show[mdl] for mdl in model_data_list]
            display(model_data_ml)
            
    def Link(b):
        
        global model_ml
        global model_data_ml
        global model_data_ml_out
        global model_data_ml_dict_show
        global model_data_list
        global w_sm_out
        global w_sm
        global Model_df
        global Model_dict
        global models_names
        
        model = model_ml.value
        data = [ mml.split(" ")[0] if("\u2714" in mml) else mml for mml in model_data_ml.value]
        
        model_data = pd.read_csv("Model#Data.csv")
        
        aux_folder = listdir("Datasets/")
        if('.csv' in aux_folder):
            aux_folder.remove('.csv')
        data_folder = [d.split(".")[0] for d in aux_folder]
        
        if(model in list(model_data["Model"])):
        
            aux = []
            
            for d in data:
                if(not(d in model_data[model_data["Model"] == model]["Data"].values[0].split("%"))):
                    aux.append(d)
                    
            data = aux
            Index = model_data.loc[model_data["Model"] == model].index[0]
            model_data.loc[Index]["Data"] =  model_data[model_data["Model"] == model]["Data"][Index]+ "%"+ "%".join(data)
            model_data.to_csv("Model#Data.csv",index=False)
            
            data = model_data[model_data["Model"] == model]["Data"].values[0].split("%")

            
            with model_data_ml_out:
                
                model_data_ml_out.clear_output()
                
                for d in data_folder:
                    
                    if(d in data):
                        model_data_ml_dict_show[d] = d + " " +"\u2714" 
                        
                    else:
                        model_data_ml_dict_show[d] = d 
               
                model_data_ml.options = [model_data_ml_dict_show[mdl] for mdl in model_data_list]
                display(model_data_ml)
            
        else:
            
        
            MD_df = pd.DataFrame({"Model":list(model_data["Model"]) + [model] , 
                                 "Data":list(model_data["Data"]) + ["%".join(data)]})
            

            MD_df.to_csv("Model#Data.csv",index=False)
            
            with model_data_ml_out:
                
                model_data_ml_out.clear_output()
                
                for d in data_folder:
                    
                    if(d in data):
                        model_data_ml_dict_show[d] = d + " " +"\u2714" 
                        
                    else:
                        model_data_ml_dict_show[d] = d  
                
               
                model_data_ml.options = [model_data_ml_dict_show[mdl] for mdl in model_data_list]
                display(model_data_ml)
        
        
       
            
            
        with w_sm_out:

            w_sm_out.clear_output()

            Model_df = pd.read_csv("Model#Data.csv")
            Model_dict = {md:dt.split("%")[0] for md,dt in zip(Model_df.Model,Model_df.Data)}
            models_names = list(Model_df.Model)

            w_sm = widgets.SelectMultiple(
                options=models_names + [''],
                value=[''] ,
                description='Models:',
                layout=widgets.Layout(width="420px",height="140px"),
                disabled=False
            ) 

            display(w_sm)
                
            
            
                
    def Link_2(b):
        
        global emb_ml
        global emb_data_ml
        global emb_data_ml_out
        global data_ml_dict_show
        global data_list
        global text_dataset_out
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        
        emb = emb_ml.value
        data = [ eml.split(" ")[0] if("\u2714" in eml) else eml for eml in emb_data_ml.value]
     
        emb_data = pd.read_csv("Emb#Data.csv")
        
        aux_folder = listdir("Trajectories/")
        if('.csv' in aux_folder):
            aux_folder.remove('.csv')
        data_folder = [d.split(".")[0] for d in aux_folder]
        
        if(emb in list(emb_data["Emb"])):
        
            aux = []
            for d in data:
                if(not(d in emb_data[emb_data["Emb"] == emb]["Data"].values[0].split("%"))):
                    aux.append(d)
                    
            data = aux
            Index = emb_data.loc[emb_data["Emb"] == emb].index[0]
            emb_data.loc[Index]["Data"] =  emb_data[emb_data["Emb"] == emb]["Data"][Index]+ "%"+ "%".join(data)
            emb_data.to_csv("Emb#Data.csv",index=False)
            
            data = emb_data[emb_data["Emb"] == emb]["Data"].values[0].split("%")
            
            with emb_data_ml_out:
                
                emb_data_ml_out.clear_output()
                
                for d in data_folder:
                    
                    if(d in data):
                        data_ml_dict_show[d] = d + " " +"\u2714" 
                        
                    else:
                        data_ml_dict_show[d] = d 
                    
                emb_data_ml.options = [data_ml_dict_show[dl] for dl in data_list]
                display(emb_data_ml)
            
        else:
            
        
            EB_df = pd.DataFrame({"Emb":list(emb_data["Emb"]) + [emb] , 
                                 "Data":list(emb_data["Data"]) + ["%".join(data)]})
            

            EB_df.to_csv("Emb#Data.csv",index=False)
            
            with emb_data_ml_out:
                
                emb_data_ml_out.clear_output()
                
                for d in data_folder:
                    
                    if(d in data):
                        data_ml_dict_show[d] = d + " " +"\u2714" 
                        
                    else:
                        data_ml_dict_show[d] = d  
                
                emb_data_ml.options = [data_ml_dict_show[dl] for dl in data_list]
                display(emb_data_ml)
                
            
        with model_choice_out:
            
            Emb_list = list(pd.read_csv("Emb.csv").Emb)
            Emb_df = pd.read_csv("Emb#Data.csv") 
            Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
            model_choice.options = Emb_list 
            model_choice.value = Emb_list[0]
            model_choice_out.clear_output()
            display(model_choice)
            
        with text_dataset_out:
            
            text_dataset_out.clear_output()
            
            text_dataset.options =  [''] if(not(Emb_list[0] in Emb_dict.keys())) else Emb_dict[Emb_list[0]] + ['']
            text_dataset.value = ''
            
            display(text_dataset)
            
            
            
            
    def Unlink(b):
        
        global model_ml
        global model_data_ml
        global model_data_ml_out
        global model_data_ml_dict_show
        global model_data_list
        global w_sm_out
        global w_sm
        global Model_df
        global Model_dict
        global models_names
        
        
        model = model_ml.value
        data = [ mml.split(" ")[0] if("\u2714" in mml) else mml for mml in model_data_ml.value]
        
        model_data = pd.read_csv("Model#Data.csv")
        
        with_remove = copy(model_data[model_data["Model"] == model]["Data"].values[0].split("%")) 
        
        for d in data:
            
            with_remove.remove(d)
            
        if(len(with_remove)>0):
            
            Index = model_data.loc[model_data["Model"] == model].index[0]
            model_data.loc[Index]["Data"] =  "%".join(with_remove)
            model_data.to_csv("Model#Data.csv",index=False)
            
        else:
            
            model_data = model_data.drop(model_data[model_data.Model == model].index)
            model_data.to_csv("Model#Data.csv",index=False)
            

        with model_data_ml_out:

            model_data_ml_out.clear_output()

            for d in data:
                model_data_ml_dict_show[d] = d

            model_data_ml.options = [model_data_ml_dict_show[mdl] for mdl in model_data_list]
            display(model_data_ml)
            
        with w_sm_out:

            w_sm_out.clear_output()

            Model_df = pd.read_csv("Model#Data.csv")
            Model_dict = {md:dt.split("%")[0] for md,dt in zip(Model_df.Model,Model_df.Data)}
            models_names = list(Model_df.Model)

            w_sm = widgets.SelectMultiple(
                options=models_names + [''],
                value=[''] ,
                description='Models:',
                layout=widgets.Layout(width="420px",height="140px"),
                disabled=False
            ) 

            display(w_sm)
            
    def Unlink_2(b):
        
        global emb_ml
        global emb_data_ml
        global emb_data_ml_out
        global data_ml_dict_show
        global data_list
        global text_dataset_out
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        emb = emb_ml.value
        data = [ eml.split(" ")[0] if("\u2714" in eml) else eml for eml in emb_data_ml.value]
        
        emb_data = pd.read_csv("Emb#Data.csv")
        
        with_remove = copy(emb_data[emb_data["Emb"] == emb]["Data"].values[0].split("%")) 
        
        for d in data:
            
            with_remove.remove(d)
            
        if(len(with_remove)>0):
            
            Index = emb_data.loc[emb_data["Emb"] == emb].index[0]
            emb_data.loc[Index]["Data"] =  "%".join(with_remove)
            emb_data.to_csv("Emb#Data.csv",index=False)
            
        else:
            
            emb_data = emb_data.drop(emb_data[emb_data.Emb == emb].index)
            emb_data.to_csv("Emb#Data.csv",index=False)
            

        with emb_data_ml_out:

            emb_data_ml_out.clear_output()

            for d in data:
                data_ml_dict_show[d] = d

            emb_data_ml.options = [data_ml_dict_show[dl] for dl in data_list]
            display(emb_data_ml)
            
        with model_choice_out:
            
            Emb_list = list(pd.read_csv("Emb.csv").Emb)
            Emb_df = pd.read_csv("Emb#Data.csv") 
            Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
            model_choice.options = Emb_list 
            model_choice.value = Emb_list[0]
            model_choice_out.clear_output()
            display(model_choice)
            
        with text_dataset_out:
            
            text_dataset_out.clear_output()
            
            text_dataset.options =  [''] if(not(Emb_list[0] in Emb_dict.keys())) else Emb_dict[Emb_list[0]] + ['']
            text_dataset.value = ''
            
            display(text_dataset)
    
            
        
            
    def Remove(b):
        
        global model_ml_up
        global model_data_ml
        global model_data_ml_out
        global model_list 
        global model_ml
        global w_sm_out
        global w_sm
        global Model_df
        global Model_dict
        global models_names
        
        
        model_data = pd.read_csv("Model#Data.csv")
        models_df = pd.read_csv("Model.csv")
        
        mmlup_val = model_ml_up.value
        aux = []
        
        for ml in model_list:
            if(not(ml in mmlup_val)):
                aux.append(ml)
                
        model_list = aux
        
        with model_ml_up_out:
            
            model_ml_up_out.clear_output()
            
            for mv in mmlup_val: 
                
                os.remove("Models/"+mv+".h5")
                models_df = models_df.drop(models_df[models_df.Model == mv].index)
                model_data = model_data.drop(model_data[model_data.Model == mv].index)
                
            
            models_df.to_csv("Model.csv",index=False)
            model_data.to_csv("Model#Data.csv",index=False)
            
            if(len(model_list)>0):
                 
                model_ml_up = widgets.SelectMultiple(options=model_list,description='Models:',
                                         value=[model_list[0]],layout=widgets.Layout(width="380px",height="180px"))
                
            else:
                 
                model_ml_up = widgets.SelectMultiple(options=model_list,description='Models:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
            
            display(model_ml_up)
            
        with model_ml_out:

            model_ml_out.clear_output()
            model_ml.options = model_list + ['']
            model_ml.value = ''
            
            display(model_ml)
            
        with w_sm_out:

            w_sm_out.clear_output()

            Model_df = pd.read_csv("Model#Data.csv")
            Model_dict = {md:dt.split("%")[0] for md,dt in zip(Model_df.Model,Model_df.Data)}
            models_names = list(Model_df.Model)

            w_sm = widgets.SelectMultiple(
                options=models_names + [''],
                value=[''] ,
                description='Models:',
                layout=widgets.Layout(width="420px",height="140px"),
                disabled=False
            ) 

            display(w_sm)
            
    def Remove_2(b):
        
        global model_data_list 
        global model_data_up_out
        global model_data_up
        global model_data_ml_dict_show
        global on_change_model_data_ml
        global w_sm_out
        global w_sm
        global Model_df
        global Model_dict
        global models_names
        
        model_data = pd.read_csv("Model#Data.csv")
        
        mdup_val = model_data_up.value
        aux = []
        
        for md in model_data_list:
            if(not(md in mdup_val)):
                aux.append(md)
                
        model_data_list = aux
        
        with model_data_up_out:
            
            model_data_up_out.clear_output()
            
            for md in mdup_val: 
                
                os.remove("Datasets/"+md+".csv")
                
                for ID in model_data.index:
                       
                    L_data = model_data.loc[ID].Data.split("%")
                    
                    if(md in L_data):
                        
                        L_data.remove(md)
                        
                        if(len(L_data)==0):
                            
                            model_data = model_data.drop(model_data[model_data.Model == model_data.loc[ID].Model].index)
                            
                        else:
                            
                            model_data.loc[ID,"Data"] = "%".join(L_data)
                
                
            
            model_data.to_csv("Model#Data.csv",index=False)
            
            if(len(model_data_list)>0):
    
                model_data_up = widgets.SelectMultiple(options=model_data_list,description='Models:',
                                         value=[model_data_list[0]],layout=widgets.Layout(width="380px",height="180px"))
            else:
        
                 model_data_up = widgets.SelectMultiple(options=model_data_list,description='Models:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
            
            display(model_data_up)
            
        with model_data_ml_out:

            model_data_ml_out.clear_output()
            
            for ex in mdup_val:
                model_data_ml_dict_show.pop(ex)
            
     
            model_data_ml = widgets.SelectMultiple(options=[model_data_ml_dict_show[mdl] for mdl in model_data_list],description='Data:',
                                             layout=widgets.Layout(width="380px",height="180px"))
            model_data_ml.observe(on_change_model_data_ml, names = 'value')
            model_data_ml.value = []
            display(model_data_ml)
            
        with w_sm_out:

            w_sm_out.clear_output()

            Model_df = pd.read_csv("Model#Data.csv")
            Model_dict = {md:dt.split("%")[0] for md,dt in zip(Model_df.Model,Model_df.Data)}
            models_names = list(Model_df.Model)

            w_sm = widgets.SelectMultiple(
                options=models_names + [''],
                value=[''] ,
                description='Models:',
                layout=widgets.Layout(width="420px",height="140px"),
                disabled=False
            ) 

            display(w_sm)
            
            
    def Remove_3(b):
        
        global data_list 
        global data_up_out
        global data_up
        global data_ml_dict_show
        global on_change_em_ml
        global text3_out
        global text_dataset_out
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        emb_data = pd.read_csv("Emb#Data.csv")
        
        dup_val = data_up.value
        aux = []
        
        for d in data_list:
            if(not(d in dup_val)):
                aux.append(d)
          
        data_list = aux
        
        with data_up_out:
            
            data_up_out.clear_output()
            
            for d in dup_val: 
                
                os.remove("Trajectories/"+d+".csv")
                    
                for ID in emb_data.index:
                       
                    L_data = emb_data.loc[ID].Data.split("%")
                    
                    if(d in L_data):
                        
                        L_data.remove(d)
                        
                        if(len(L_data)==0):
                            
                            emb_data = emb_data.drop(emb_data[emb_data.Emb == emb_data.loc[ID].Emb].index)
                            
                        else:
                            
                            emb_data.loc[ID,"Data"] = "%".join(L_data)
                
                
            
            emb_data.to_csv("Emb#Data.csv",index=False)
                     
            
            if(len(data_list)>0):
    
                data_up = widgets.SelectMultiple(options=data_list,description='Data:',
                                         value=[data_list[0]],layout=widgets.Layout(width="380px",height="180px"))
            else:
        
                 data_up = widgets.SelectMultiple(options=data_list,description='Data:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
            
            display(data_up)
            
        with emb_data_ml_out:

            emb_data_ml_out.clear_output()
            
            for ex in dup_val:
                data_ml_dict_show.pop(ex)
                
            
            emb_data_ml = widgets.SelectMultiple(options=[data_ml_dict_show[dl] for dl in data_list],description='Data:',
                                             layout=widgets.Layout(width="380px",height="180px"))
            emb_data_ml.observe(on_change_emb_data_ml, names = 'value')
            emb_data_ml.value = []
            display(emb_data_ml)
            
        with text3_out:

            text3_out.clear_output()
            text3 = widgets.Dropdown(description='',layout=widgets.Layout(width="80px"),options=data_list)
            text3.observe(on_change_dataset, names='value')
            display(text3)
            
        with model_choice_out:
            
            Emb_list = list(pd.read_csv("Emb.csv").Emb)
            Emb_df = pd.read_csv("Emb#Data.csv") 
            Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
            model_choice.options = Emb_list 
            model_choice.value = Emb_list[0]
            model_choice_out.clear_output()
            display(model_choice)
            
        with text_dataset_out:
            
            text_dataset_out.clear_output()
            
            text_dataset.options =  [''] if(not(Emb_list[0] in Emb_dict.keys())) else Emb_dict[Emb_list[0]] + ['']
            text_dataset.value = ''
            
            display(text_dataset)
            
    def Remove_4(b):
  
        global emb_ml_up
        global emb_list 
        global emb_ml
        global on_change_emb_ml
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        emb_data = pd.read_csv("Emb#Data.csv")
        emb_df = pd.read_csv("Emb.csv")
        
        emlup_val = emb_ml_up.value
        aux = []
        for emb in emb_list:
            if(not(emb in emlup_val)):
                aux.append(emb)
                
        
        emb_list = aux
        
        with emb_ml_up_out:
            emb_ml_up_out.clear_output()
            
            for emb in emlup_val: 
                
                os.remove("Embeddings/"+emb+".csv")
                emb_df = emb_df.drop(emb_df[emb_df.Emb == emb].index)
                emb_data = emb_data.drop(emb_data[emb_data.Emb == emb].index)
                
            
            emb_df.to_csv("Emb.csv",index=False)
            emb_data.to_csv("Emb#Data.csv",index=False)
            
            
            
            if(len(emb_list)>0):
                emb_ml_up = widgets.SelectMultiple(options=emb_list,description='Emb:',
                                         value=[emb_list[0]],layout=widgets.Layout(width="380px",height="180px"))
                
            else:
                emb_ml_up = widgets.SelectMultiple(options=emb_list,description='Emb:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
            
            display(emb_ml_up)
            
        with emb_ml_out:

            emb_ml_out.clear_output()
            emb_ml.options = emb_list + ['']
            emb_ml.value = ''
            
            display(emb_ml)
            
            
        with model_choice_out:
            
            Emb_list = list(pd.read_csv("Emb.csv").Emb)
            Emb_df = pd.read_csv("Emb#Data.csv") 
            Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
            model_choice.options = Emb_list 
            model_choice.value = Emb_list[0]
            model_choice_out.clear_output()
            display(model_choice)
            
    def Name(b):
        
        global model_ml_up
        global model_list 
        global model_ml
        global ul_2
        global w_sm_out
        global w_sm
        global Model_df
        global Model_dict
        global models_names
       
        
        model_data = pd.read_csv("Model#Data.csv")
        models_df = pd.read_csv("Model.csv")
        
        mmlup_val = model_ml_up.value
        aux = []
        
        
        for ml in model_list:#
            if(ml in mmlup_val):
                aux.append(ul_2)
            else:
                aux.append(ml)
                
        model_list = aux
        
        with model_ml_up_out:
            
            model_ml_up_out.clear_output()
            
            for mv in mmlup_val: 
                
                os.rename("Models/"+mv+".h5", "Models/"+ul_2+".h5")
                models_df.replace(mv,ul_2,inplace=True)
                model_data.replace(mv,ul_2,inplace=True)
                
            
            models_df.to_csv("Model.csv",index=False)
            model_data.to_csv("Model#Data.csv",index=False)
                 
    
            if(len(model_list)>0):
    
                model_ml_up = widgets.SelectMultiple(options=model_list,description='Models:',
                                         value=[model_list[0]],layout=widgets.Layout(width="380px",height="180px"))
            else:
        
                model_ml_up = widgets.SelectMultiple(options=model_list,description='Models:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
            
            display(model_ml_up)
            
        with model_ml_out:

            model_ml_out.clear_output()
            model_ml = widgets.Select(options=model_list + [''],description='Models:',value='',layout=widgets.Layout(width="380px",height="180px"))
            model_ml.observe(on_change_model_ml, names = 'value')
            model_ml.value = model_list[0]
            display(model_ml)
            
        with w_sm_out:

            w_sm_out.clear_output()

            Model_df = pd.read_csv("Model#Data.csv")
            Model_dict = {md:dt.split("%")[0] for md,dt in zip(Model_df.Model,Model_df.Data)}
            models_names = list(Model_df.Model)

            w_sm = widgets.SelectMultiple(
                options=models_names + [''],
                value=[''] ,
                description='Models:',
                layout=widgets.Layout(width="420px",height="140px"),
                disabled=False
            ) 

            display(w_sm)
            
        
    def Name_2(b):
        
        global model_data_list 
        global model_data_up_out
        global model_data_up
        global model_data_ml_dict_show
        global on_change_model_data_ml
        global model_data_list
        global w_sm_out
        global w_sm
        global Model_df
        global Model_dict
        global models_names
        global ul_3
        
        model_data = pd.read_csv("Model#Data.csv")
        
        mdup_val = model_data_up.value
        aux = []
        
        for md in model_data_list:
            if(md in mdup_val):
                aux.append(ul_3)
            else:
                aux.append(md)
                
        model_data_list = aux
        
        with model_data_up_out:
            
            model_data_up_out.clear_output()
            
            for md in mdup_val: 
                
                os.rename("Datasets/"+md+".csv","Datasets/"+ul_3+".csv")
                
                for ID in model_data.index:
                    
                    L_data = model_data.loc[ID].Data.split("%")
                    
                    aux_ld =[]
                    
                    for ld in L_data:
                    
                        if(ld==md):
                            aux_ld.append(ul_3)
                        else:
                            aux_ld.append(ld)
                        
                    L_data = aux_ld                   
                            
                    model_data.loc[ID,"Data"] = "%".join(L_data)
                
                
            
            model_data.to_csv("Model#Data.csv",index=False)
            
            if(len(model_data_list)>0):
    
                model_data_up = widgets.SelectMultiple(options=model_data_list,description='Data:',
                                         value=[model_data_list[0]],layout=widgets.Layout(width="380px",height="180px"))
            else:
        
                 model_data_up = widgets.SelectMultiple(options=model_data_list,description='Data:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
            
            display(model_data_up)
            
        with model_data_ml_out:

            model_data_ml_out.clear_output()
            
            for ex in mdup_val:
                
                if("\u2714" in model_data_ml_dict_show[ex]):
                    
                    model_data_ml_dict_show[ul_3] = ul_3 + " " +"\u2714" 
                    
                else:
                    
                    model_data_ml_dict_show[ul_3] = ul_3
                    
                model_data_ml_dict_show.pop(ex)

            model_data_ml = widgets.SelectMultiple(options=[model_data_ml_dict_show[mdl] for mdl in model_data_list],description='Data:',
                                             layout=widgets.Layout(width="380px",height="180px"))
            model_data_ml.observe(on_change_model_data_ml, names = 'value')
            display(model_data_ml)
            
        with w_sm_out:

            w_sm_out.clear_output()

            Model_df = pd.read_csv("Model#Data.csv")
            Model_dict = {md:dt.split("%")[0] for md,dt in zip(Model_df.Model,Model_df.Data)}
            models_names = list(Model_df.Model)

            w_sm = widgets.SelectMultiple(
                options=models_names + [''],
                value=[''] ,
                description='Models:',
                layout=widgets.Layout(width="420px",height="140px"),
                disabled=False
            ) 

            display(w_sm)
         
    
    def Name_3(b):
    
        global data_list 
        global data_up_out
        global data_up
        global emb_data_ml_out
        global data_ml_dict_show
        global text3_out
        global on_change_emb_ml
        global ul
        global text_dataset_out
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        emb_data = pd.read_csv("Emb#Data.csv")
        
        dup_val = data_up.value
        aux = []
        
        for d in data_list:
            if(d in dup_val):
                aux.append(ul)
            else:
                aux.append(d)
                
        data_list = aux
                
        
        with data_up_out:
            
            data_up_out.clear_output()
            
            for d in dup_val:
                
                os.rename("Trajectories/"+d+".csv","Trajectories/"+ul+".csv")
            
                for ID in emb_data.index:

                    L_data = emb_data.loc[ID].Data.split("%")

                    aux_ld =[]

                    for ld in L_data:

                        if(ld==d):
                            aux_ld.append(ul)
                        else:
                            aux_ld.append(ld)

                    L_data = aux_ld                   

                    emb_data.loc[ID,"Data"] = "%".join(L_data)



            emb_data.to_csv("Emb#Data.csv",index=False)
                
            
            if(len(data_list)>0):
    
                data_up = widgets.SelectMultiple(options=data_list,description='Data:',
                                         value=[data_list[0]],layout=widgets.Layout(width="380px",height="180px"))
            else:
        
                 data_up = widgets.SelectMultiple(options=data_list,description='Data:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
            
            display(data_up)
            
        with emb_data_ml_out:

            emb_data_ml_out.clear_output()
            
            for ex in dup_val:
                
                if("\u2714" in data_ml_dict_show[ex]):
                    
                    data_ml_dict_show[ul] = ul  + " " +"\u2714" 
                    
                else:
                    
                    data_ml_dict_show[ul] = ul
                    
                data_ml_dict_show.pop(ex)

            emb_data_ml = widgets.SelectMultiple(options=[data_ml_dict_show[edl] for edl in data_list],description='Data:',
                                             layout=widgets.Layout(width="380px",height="180px"))
            emb_data_ml.observe(on_change_emb_data_ml, names = 'value')
            display(emb_data_ml) 
            
        with text3_out:

            text3_out.clear_output()
            text3 = widgets.Dropdown(description='',layout=widgets.Layout(width="80px"),options=data_list)
            text3.observe(on_change_dataset, names='value')
            display(text3)
            
        with model_choice_out:
            
            Emb_list = list(pd.read_csv("Emb.csv").Emb)
            Emb_df = pd.read_csv("Emb#Data.csv") 
            Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
            model_choice.options = Emb_list 
            model_choice.value = Emb_list[0]
            model_choice_out.clear_output()
            display(model_choice)
            
        with text_dataset_out:
            
            text_dataset_out.clear_output()
            
            text_dataset.options =  [''] if(not(Emb_list[0] in Emb_dict.keys())) else Emb_dict[Emb_list[0]] + ['']
            text_dataset.value = ''
            
            display(text_dataset)
            
    def Name_4(b):    
                    
        global emb_ml_up
        global emb_list 
        global emb_ml
        global on_change_emb_ml
        global ul_4
        global model_choice_out
        global Emb_list
        global Emb_df
        global Emb_dict
        
        emb_data = pd.read_csv("Emb#Data.csv")
        emb_df = pd.read_csv("Emb.csv")
        
        emlup_val = emb_ml_up.value
        aux = []
        
        
        for emb in emb_list:
            if(emb in emlup_val):
                aux.append(ul_4)
            else:
                aux.append(emb)
                
        emb_list = aux
        
        with emb_ml_up_out:
            
            emb_ml_up_out.clear_output()
            
            for emb in emlup_val: 
                
                os.rename("Embeddings/"+emb+".csv", "Embeddings/"+ul_4+".csv")
                emb_df.replace(emb,ul_4,inplace=True)
                emb_data.replace(emb,ul_4,inplace=True)
                
            
            emb_df.to_csv("Emb.csv",index=False)
            emb_data.to_csv("Emb#Data.csv",index=False)
            
            if(len(emb_list)>0):
    
                emb_ml_up = widgets.SelectMultiple(options=emb_list,description='Emb:',
                                         value=[emb_list[0]],layout=widgets.Layout(width="380px",height="180px"))
            else:
        
                 emb_ml_up = widgets.SelectMultiple(options=emb_list,description='Emb:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
                 
            
            display(emb_ml_up)
            
        with emb_ml_out:

            emb_ml_out.clear_output()
            emb_ml = widgets.Select(options=emb_list + [''],description='Embeddings:',value='',layout=widgets.Layout(width="380px",height="180px"))
            emb_ml.observe(on_change_emb_ml, names = 'value')
            emb_ml.value = emb_list[0]
            display(emb_ml)
            
            
        with model_choice_out:
            
            Emb_list = list(pd.read_csv("Emb.csv").Emb)
            Emb_df = pd.read_csv("Emb#Data.csv") 
            Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
            model_choice.options = Emb_list 
            model_choice.value = Emb_list[0]
            model_choice_out.clear_output()
            display(model_choice)
       
        
        
    def on_change_datasets_2(change):

        global dataset_4
        dataset_4 = change.new
        
    def on_change_model_data_ml(change):
        
        global md_dt_ml
        md_dt_ml = change.new
        
        
    def on_change_emb_data_ml(change):
        
        global emb_dt_ml
        emb_dt_ml = change.new
        
    def on_change_ul(change):

        global ul
        ul = change.new
        
    def on_change_ul_2(change):

        global ul_2
        ul_2 = change.new
        
    def on_change_ul_3(change):

        global ul_3
        ul_3 = change.new
        
    def on_change_ul_4(change):

        global ul_4
        ul_4 = change.new
        
        
    def on_change_model_ml(change):
        
        global md_ml
        global model_data_ml
        global model_data_ml_out
        global model_data_list
        global model_data_ml_dict_show 
        md_ml = change.new
        
        model_data = pd.read_csv("Model#Data.csv")
        
        
        if(md_ml in list(model_data["Model"])):
            
            data_link_string = model_data[model_data["Model"] == md_ml]["Data"].values[0].split("%")
            aux_folder = listdir("Datasets/")
            if('.csv' in aux_folder):
                aux_folder.remove('.csv')
            data_folder = [data.split(".")[0] for data in aux_folder]

            with model_data_ml_out:

                model_data_ml_out.clear_output()
                
                for data in data_folder:
                    
                    if(data in data_link_string):
                    
                        model_data_ml_dict_show[data] = data + " " +"\u2714" 
                    
                    else:
                        
                        model_data_ml_dict_show[data] = data
                
                model_data_ml.options = [model_data_ml_dict_show[mdl] for mdl in model_data_list]
                display(model_data_ml)
            
        else:

            with model_data_ml_out:

                model_data_ml_out.clear_output()
                model_data_ml.options = [mdl for mdl in model_data_list]
                
                for mdl in model_data_list:
                    model_data_ml_dict_show[mdl] = mdl
                
                display(model_data_ml)
                
    def on_change_emb_ml(change): 
        
        global eb_ml
        global emb_data_ml
        global emb_data_ml_out
        global data_list
        global data_ml_dict_show 
        global emb_list
        
        eb_ml = change.new
        
        
        emb_data = pd.read_csv("Emb#Data.csv")
        
        
        if(eb_ml in list(emb_data["Emb"])):
            
            data_link_string = emb_data[emb_data["Emb"] == eb_ml]["Data"].values[0].split("%")
            aux_folder = listdir("Trajectories/")
            if('.csv' in aux_folder):
                aux_folder.remove('.csv')
            data_folder = [data.split(".")[0] for data in aux_folder]

            with emb_data_ml_out:

                emb_data_ml_out.clear_output()
                
                for data in data_folder:
                    
                    if(data in data_link_string):
                    
                        data_ml_dict_show[data] = data + " " +"\u2714" 
                    
                    else:
                        
                        data_ml_dict_show[data] = data
                
                emb_data_ml.options = [data_ml_dict_show[dl] for dl in data_list]
                display(emb_data_ml)
            
        else:

            with emb_data_ml_out:

                emb_data_ml_out.clear_output()
                emb_data_ml.options = [edl for edl in data_list]
                display(emb_data_ml)
    
    
        
        
    
    spc = widgets.Label("")
    spc1 = widgets.Label("",layout=widgets.Layout(width="9%"))
    spc2 = widgets.Label("",layout=widgets.Layout(width="6%"))
    
    ul = "" 
    ul_2 = ""
    ul_3 = ""
    ul_4 = ""
    
    md_ml = ""
    md_dt_ml = ""
    eb_ml = ""
    emb_dt_ml = ""
    
    model_data_list = [model_data.split(".")[0] for model_data in listdir("Datasets/")]
    model_list = [model.split(".")[0] for model in listdir("Models/")]
    data_list = [dt.split(".")[0] for dt in listdir("Trajectories/")]
    emb_list = [eb.split(".")[0] for eb in listdir("Embeddings/")]
   
    
    if("" in data_list):
        data_list.remove("")
    
    if("" in model_data_list):
        model_data_list.remove("")
        
    if("" in data_list):
        data_list.remove("")
        
    if("" in emb_list):
        emb_list.remove("")

    model_data_ml_dict_show = {mdl:mdl for mdl in model_data_list}
    data_ml_dict_show = {dl:dl for dl in data_list}

    b1 = widgets.FileUpload(acept=".csv",multiple=True,layout=widgets.Layout(width="150px"))
    b1.observe(func1,names="value")
    
    b_remove_3 = widgets.Button(description="Remove", layout=widgets.Layout(width="150px"))
    b_remove_3.on_click(Remove_3)
    
    data_up = ""
    
    if(len(data_list)>0):
    
         data_up = widgets.SelectMultiple(options=data_list,description='Data:',
                                         value=[data_list[0]],layout=widgets.Layout(width="380px",height="180px"))
    else:
        
         data_up = widgets.SelectMultiple(options=data_list,description='Data:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
    
    data_up_out = setup_ui(data_up)
    
    b_name_3 = widgets.Button(description="Rename", layout=widgets.Layout(width="90px"))
    b_name_3.style.button_color = "lightgray"
    b_name_3.on_click(Name_3)
    
    user_label = widgets.Text(description='',layout=widgets.widgets.Layout(width="250px"))
    user_label.observe(on_change_ul, names = 'value')
    
    
    
    
    h2 = widgets.HBox(children=[widgets.VBox([b1,b_remove_3,spc1]),
                                              spc1,widgets.VBox([data_up_out,spc2,
                                              widgets.HBox([spc1,spc1,spc2,user_label,b_name_3])])])
    
    
    
    
    
    
    
    
    
    
    
    
    b1_2 = widgets.FileUpload(acept=".csv",multiple=True,layout=widgets.Layout(width="150px"))
    b1_2.observe(func2,names="value")
    
    
    b_remove_4 = widgets.Button(description="Remove", layout=widgets.Layout(width="150px"))
    b_remove_4.on_click(Remove_4)
    
    
    b_name_4 = widgets.Button(description="Rename", layout=widgets.Layout(width="90px"))
    b_name_4.style.button_color = "lightgray"
    b_name_4.on_click(Name_4)
    
    user_label_4 = widgets.Text(description='',layout=widgets.widgets.Layout(width="250px"))
    user_label_4.observe(on_change_ul_4, names = 'value')
    
    
    emb_ml_up = ""
    
    
    if(len(emb_list)>0):
        emb_ml_up = widgets.SelectMultiple(options=emb_list,description='Emb:',
                                         value=[emb_list[0]],layout=widgets.Layout(width="380px",height="180px"))
    else:       
        emb_ml_up = widgets.SelectMultiple(options=emb_list,description='Emb:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
    
    emb_ml_up_out = setup_ui(emb_ml_up)
        
    
    
    h1 = widgets.HBox(children=[widgets.VBox([b1_2,b_remove_4,spc1]),
                                              spc1,widgets.VBox([emb_ml_up_out,spc2,
                                              widgets.HBox([spc1,spc1,spc2,user_label_4,b_name_4])])])
    
    tab_emb = widgets.Tab()
    
        
    
    emb_data_ml = widgets.SelectMultiple(options=[data_ml_dict_show[dl] for dl in data_list],description='Data:',
                                         layout=widgets.Layout(width="380px",height="180px"))
    emb_data_ml.observe(on_change_emb_data_ml, names = 'value')
    emb_data_ml_out = setup_ui(emb_data_ml)
    
    
    emb_ml = widgets.Select(options=emb_list + [''],description='Emb:',value='',layout=widgets.Layout(width="380px",height="180px"))
    emb_ml.observe(on_change_emb_ml, names = 'value')
    
    
    
    emb_ml_out = setup_ui(emb_ml)
    
    
    
    link_bt_2 = widgets.Button(description="Link", layout=widgets.Layout(width="80px"))
    link_bt_2.style.button_color = "lightgray"
    link_bt_2.on_click(Link_2)
    
    unlink_bt_2 = widgets.Button(description="Ulink", layout=widgets.Layout(width="80px"))
    unlink_bt_2.style.button_color = "lightgray"
    unlink_bt_2.on_click(Unlink_2)
    
    
    
    h1_2 = widgets.VBox([widgets.HBox([link_bt_2,widgets.Label("",layout=widgets.Layout(width="1%")),unlink_bt_2]), 
                         spc2,widgets.HBox([emb_ml_out,spc2,emb_data_ml_out])])
    
    tab_emb.children = [h1,h1_2]
    
    
    tab_emb.set_title(0,'Upload')
    tab_emb.set_title(1,'Data Linkage')
    
    
    
    
    
    
    
    
    
    
    
    
    b1_3 = widgets.FileUpload(acept=".h5",multiple=True,layout=widgets.Layout(width="150px"))
    b1_3.observe(func3,names="value")
    b_remove = widgets.Button(description="Remove", layout=widgets.Layout(width="150px"))
    b_remove.on_click(Remove)
    
    
    b_name = widgets.Button(description="Rename", layout=widgets.Layout(width="90px"))
    b_name.style.button_color = "lightgray"
    b_name.on_click(Name)
    
    user_label_2 = widgets.Text(description='',layout=widgets.widgets.Layout(width="250px"))
    user_label_2.observe(on_change_ul_2, names = 'value')
    
    
    model_ml_up = ""
    
    
    if(len(model_list)>0):
    
        model_ml_up = widgets.SelectMultiple(options=model_list,description='Models:',
                                         value=[model_list[0]],layout=widgets.Layout(width="380px",height="180px"))
    else:
        
         model_ml_up = widgets.SelectMultiple(options=model_list,description='Models:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
    
    model_ml_up_out = setup_ui(model_ml_up)
        
    
    
    h3 = widgets.HBox(children=[widgets.VBox([b1_3,b_remove,spc1]),
                                              spc1,widgets.VBox([model_ml_up_out,spc2,
                                              widgets.HBox([spc1,spc1,spc2,user_label_2,b_name])])])
    
    tab_model = widgets.Tab()
    
        
    
    model_data_ml = widgets.SelectMultiple(options=[model_data_ml_dict_show[mdl] for mdl in model_data_list],description='Data:',layout=widgets.Layout(width="380px",height="180px"))
    model_data_ml.observe(on_change_model_data_ml, names = 'value')
    model_data_ml_out = setup_ui(model_data_ml)
    
    model_ml = widgets.Select(options=model_list + [''],description='Models:',value='',layout=widgets.Layout(width="380px",height="180px"))
    model_ml.observe(on_change_model_ml, names = 'value')
    
    
    model_ml_out = setup_ui(model_ml)
    
    
    
    link_bt = widgets.Button(description="Link", layout=widgets.Layout(width="80px"))
    link_bt.style.button_color = "lightgray"
    link_bt.on_click(Link)
    
    unlink_bt = widgets.Button(description="Ulink", layout=widgets.Layout(width="80px"))
    unlink_bt.style.button_color = "lightgray"
    unlink_bt.on_click(Unlink)
    
    h3_2 = widgets.VBox([widgets.HBox([link_bt,widgets.Label("",layout=widgets.Layout(width="1%")),unlink_bt]), 
                         spc2,widgets.HBox([model_ml_out,spc2,model_data_ml_out])])
    
    tab_model.children = [h3,h3_2]
    
    
    tab_model.set_title(0,'Upload')
    tab_model.set_title(1,'Data Linkage')
    
    
    
    
    
    
    b1_4 = widgets.FileUpload(acept=".csv",multiple=True,layout=widgets.Layout(width="150px"))
    b1_4.observe(func4,names="value")
    
    b_remove_2 = widgets.Button(description="Remove", layout=widgets.Layout(width="150px"))
    b_remove_2.on_click(Remove_2)
    
    model_data_up = ""
    
    if(len(model_data_list)>0):
    
        model_data_up = widgets.SelectMultiple(options=model_data_list,description='Models:',
                                         value=[model_data_list[0]],layout=widgets.Layout(width="380px",height="180px"))
    else:
        
         model_data_up = widgets.SelectMultiple(options=model_data_list,description='Models:',
                                         value=[],layout=widgets.Layout(width="380px",height="180px"))
    
    model_data_up_out = setup_ui(model_data_up)
    
    b_name_2 = widgets.Button(description="Rename", layout=widgets.Layout(width="90px"))
    b_name_2.style.button_color = "lightgray"
    b_name_2.on_click(Name_2)
    user_label_3 = widgets.Text(description='',layout=widgets.widgets.Layout(width="250px"))
    user_label_3.observe(on_change_ul_3, names = 'value')
    
    h4 = widgets.HBox(children=[widgets.VBox([b1_4,b_remove_2,spc1]),
                                              spc1,widgets.VBox([model_data_up_out,spc2,
                                              widgets.HBox([spc1,spc1,spc2,user_label_3,b_name_2])])])
    
    
    accordion5 = widgets.Accordion(children=[h2],selected_index=None)
    accordion6 = widgets.Accordion(children=[tab_emb],selected_index=None)
    accordion7 = widgets.Accordion(children=[tab_model],selected_index=None)
    accordion8 = widgets.Accordion(children=[h4],selected_index=None)
    #accordion2 = widgets.Accordion(children=[acc_similar_loc],selected_index=None)
    
    accordion5.set_title(0, 'Datasets Import')
    accordion6.set_title(0, 'Embeddings Import')
    accordion7.set_title(0, 'Models Import')
    accordion8.set_title(0, 'Model Datasets Import')

    display(widgets.VBox([accordion5,spc,accordion8,spc,accordion6,spc,accordion7]))




with out2:
    
    global Datasets

    out_df_2 = setup_ui(widgets.Label(""))
    out_plot_2 = setup_ui(widgets.Label(""))
    out_df_2.layout = widgets.Layout(border='solid 1.5px',margin='0px 10px 10px 10px',padding='5px 5px 5px 5px')
    out_plot_2.layout = widgets.Layout(border='solid 2.0px',margin='0px 10px 10px 10px',padding='5px 5px 5px 5px')
  
    out_statistics = widgets.Output()
    out_sampling = widgets.Output()

    out_statistics.layout = widgets.Layout(border='solid 1.5px white',margin='10px 70px 10px 0px',
                                   padding='5px 5px 5px 5px')
  
    out_sampling.layout = widgets.Layout(border='solid 1.5px white',margin='20px 10px 10px 0px',
                                   padding='5px 5px 5px 5px')

    DatasetsList = [dataframe.split(".")[0] for dataframe in listdir("Trajectories/")]               
    dataset_2  = DatasetsList[0]
    sampling = 10

    def Show_Statistics(b):

        with out_df_2:

            global dataset_2
            DF = pd.read_csv("Trajectories/"+dataset_2+".csv")
            
            if(len(DF) > 0):
            
                out_df_2.clear_output()
                
                
                number_of_trajectories = len(set(DF["trajectory_id"]))
                length_trajectories = [len(DF[DF["trajectory_id"] == tj]) for tj in set(DF["trajectory_id"])]
                maximum_length_of_trajectories = max(length_trajectories)
                minimum_length_of_trajectories = min(length_trajectories)
                average_length_of_trajectories = int(sum(length_trajectories)/len(length_trajectories))

                df1 = ""
        
                if("location_label" in list(DF.columns)):
                    number_of_distinct_locations = len(set(DF["location_label"]))
                    df1 = pd.DataFrame({" ":[number_of_trajectories,maximum_length_of_trajectories,minimum_length_of_trajectories,average_length_of_trajectories,
                                  number_of_distinct_locations]}, index = ["number_of_trajectories","maximum_length_of_trajectories","minimum_length_of_trajectories",
                                                                          "average_length_of_trajectories","number_of_distinct_locations"])
                else:
                    df1 = pd.DataFrame({" ":[number_of_trajectories,maximum_length_of_trajectories,minimum_length_of_trajectories,average_length_of_trajectories]}, 
                                     index = ["number_of_trajectories","maximum_length_of_trajectories","minimum_length_of_trajectories",
                                                                          "average_length_of_trajectories"])


                display(df1)

            else:
                out_df_2.clear_output()
                display(widgets.Label("Dataset Vazio !"))

    def Plot_Sampling(b):

        with out_plot_2:

            global sampling
            DF = pd.read_csv("Trajectories/"+dataset_2+".csv")
        
            if(len(DF) > 0):

                out_plot_2.clear_output()
                set_traj = set(DF["trajectory_id"])

                # Setar o Trjaectory_ID pra set o index
                # Setar no pymove, Filter byID

                Sampling_Traj = sample(set_traj,int(sampling))
                DataFrame = pd.concat([filters.by_label(DF, value = tj, label_name = "trajectory_id") for tj in Sampling_Traj],axis=0)
        

                move_df = MoveDataFrame(data=DataFrame, latitude="lat", longitude="lon", datetime="time", 
                                        traj_id='trajectory_id')
                figure = folium.Figure(width=800, height=300) 

                display(f.plot_trajectories(move_df).add_to(figure))

            else:

                out_plot_2.clear_output()
                display(widgets.Label("Dataset Vazio !"))

  
    def on_change_dataset(change):

        global dataset_2
        dataset_2 = change.new

    def on_change_sampling(change):

        global sampling
        sampling = change.new

    with out_statistics:
        
        text2 = widgets.Label("Trajectory Data")
        spc = widgets.Label("")
        text3 = widgets.Dropdown(description='',layout=widgets.Layout(width="80px"),options=DatasetsList)
        text3.observe(on_change_dataset, names='value')
        text3_out = setup_ui(text3)
        b3 = widgets.Button(description="Show Statistics", layout=widgets.Layout(width="120px"))
        b3.on_click(Show_Statistics)
        b3.style.button_color = "lightgray"

        h4 = widgets.VBox(children=[text2,widgets.HBox(children=[spc,text3_out,spc,b3])])
        h5 = widgets.VBox(children=[h4,widgets.Label("",layout=widgets.Layout(width="19%")),out_df_2])

        display(h5) 

    with out_sampling: 
        
        text_smp = widgets.Label("Sampling")
        text10 = widgets.Text(description='',layout=widgets.widgets.Layout(width="80px"))
        text10.observe(on_change_sampling, names = 'value')
        b4 = widgets.Button(description="Plot Trajectories",layout=widgets.Layout(width="120px"))
        b4.style.button_color = "lightgray"
        b4.on_click(Plot_Sampling)
        h8 = widgets.VBox(children=[text_smp,widgets.HBox(children=[text10,b4])])

        h9 = widgets.VBox(children=[h8,spc,out_plot_2])

        display(h9)

    h10 = widgets.HBox(children=[out_statistics,out_sampling])

    display(h10)
  
  



with out4:
    
    output = widgets.Output()
    spc = widgets.Label(" ")
    spc1 = widgets.Label("",layout=widgets.Layout(width="102px"))
    
    def on_button_clicked(b):
        
       
        
        with output:
            
            output.clear_output()
            
            global w_sm
            
            w_sm_vals = w_sm.value
            list_out = []

            for i in range(len(w_sm_vals)):
                list_out.append(extrinsic_evaluation(w_sm_vals[i],Model_dict))

            df_out = pd.DataFrame(list_out, columns=['Model', 'Acc@1', 'Acc@3', 'Acc@5', 'F1-score'])
            df_out = df_out.style.highlight_max(axis=0, subset=['Acc@1','Acc@3','Acc@5', 'F1-score'], props='font-weight:bold;')
            df_out.format({'Acc@1': "{:.3}", 'Acc@3': "{:.3}", 'Acc@5': "{:.3}", 'F1-score': "{:.3}"})
            df_out.hide_index()
            display(df_out)

    
    
    Model_df = pd.read_csv("Model#Data.csv")
    Model_dict = {md:dt.split("%")[0] for md,dt in zip(Model_df.Model,Model_df.Data)}
    models_names = list(Model_df.Model)
    
    
    w_sm = widgets.SelectMultiple(
        options=models_names + [''],
        value=[''] ,
        description='Models:',
        layout=widgets.Layout(width="420px",height="140px"),
        disabled=False
    )  
    
    w_sm_out = setup_ui(w_sm)
    
    
    bt = widgets.Button(description="Evaluation", layout=widgets.Layout(width="100px"))
    bt.style.button_color = "lightgray"
    bt.on_click(on_button_clicked)
    display(widgets.VBox([widgets.HBox([w_sm_out,spc,spc,spc,bt]),spc,widgets.HBox([spc1,output])]))

    

   
    
with out3:
    
    Labels = {"External Sensors Trajectories":"test_trajs.txt"}

    html1 = "<h1 style='font-size:15px'> MRR  </h1>"
    html2 = "<h1 style='font-size:15px'> Similar Locations </h1>"
    html3 = "<h1 style='font-size:15px'> Similar Trajectories </h1>"
    
    
    traj_data  = "External Sensors Trajectories"
    
    Emb_list = list(pd.read_csv("Emb.csv").Emb)
    Emb_df = pd.read_csv("Emb#Data.csv") 
    Emb_dict = { emb: Emb_df[Emb_df["Emb"]==emb].Data.values[0].split("%") for emb in list(Emb_df.Emb)}
    
    Edit_Matrix = pd.read_csv("Matrix/"+"Dist_Edit_Distance.csv", index_col=0)
    DTW_Matrix = pd.read_csv("Matrix/"+"Dist_DTW_new.csv", index_col=0)
    Eucli_Matrix_sensors = pd.read_csv("Matrix/"+"Eucli_Matrix_sensors.csv",index_col=0)
    Road_Matrix_sensors = pd.read_csv("Matrix/"+"Road_Matrix_sensors.csv",index_col=0)
    DTW_Matrix.columns = [int(i) for i in DTW_Matrix.columns]
    Edit_Matrix.columns = [int(i) for i in Edit_Matrix.columns]
    Eucli_Matrix_sensors.columns = [str(i) for i in Eucli_Matrix_sensors.columns]
    Road_Matrix_sensors.columns = [str(i) for i in Road_Matrix_sensors.columns]
    
    for i in range(0,DTW_Matrix.shape[0]):

        DTW_Matrix.iloc[i,i] = np.nan
        Edit_Matrix.iloc[i,i] = np.nan
    
    Dataset = ""
    tokenizer_df = ""
    embeddding_matrix = ""
    Dataset_geo25 = ""
    Dict_Geo25 = ""
    Cossine_Matrix_sensors = ""
    CM_sensors = ""
    query_index = ""
    Objects_loc = []
    obj_loc = ""
    LatLong = ""
    
    if(len(list(Emb_df.Emb))==0):
        
        aux_emb = pd.read_csv("Embeddings/" + Emb_list[0] + ".csv") 

        tokenizer_df = aux_emb.loc[0:list(pd.isnull(aux_emb["sensor"])).index(True)-1,["sensor","id"]]
        tokenizer_df.id = [int(i) for i in tokenizer_df.id]
        tokenizer_df.index = tokenizer_df.id
        tokenizer_df = tokenizer_df[["sensor"]]

        embeddding_matrix = aux_emb.loc[list(pd.isnull(aux_emb["sensor"])).index(True):int(list(pd.isnull(aux_emb["sensor"])).index(True)*2),
                        [str(i) for i in range(len(aux_emb.columns)-2)]]

        embeddding_matrix.index = [i for i in range(embeddding_matrix.shape[0])]

    
    elif(len(list(Emb_df.Emb))>1):
    
        aux_emb = pd.read_csv("Embeddings/" + Emb_list[0] + ".csv") 

        tokenizer_df = aux_emb.loc[0:list(pd.isnull(aux_emb["sensor"])).index(True)-1,["sensor","id"]]
        tokenizer_df.id = [int(i) for i in tokenizer_df.id]
        tokenizer_df.index = tokenizer_df.id
        tokenizer_df = tokenizer_df[["sensor"]]

        embeddding_matrix = aux_emb.loc[list(pd.isnull(aux_emb["sensor"])).index(True):int(list(pd.isnull(aux_emb["sensor"])).index(True)*2),
                        [str(i) for i in range(len(aux_emb.columns)-2)]]

        embeddding_matrix.index = [i for i in range(embeddding_matrix.shape[0])]

        Dataset = pd.read_csv("Traj_Datasets/"+ Labels["External Sensors Trajectories"])

        Dataset["0"] = Dataset["0"].apply(lambda x : x.replace("[","").replace("]","").replace(",","").replace("\n","").replace("'",""))
        Dataset["0"] = Dataset["0"].apply(lambda x : x.split())
        Dataset.rename(columns = {'Unnamed: 0':'trajectory_number'}, inplace = True)
        Dataset["trajectory_number"] = Dataset["trajectory_number"].apply(lambda x: str(x))
        Dataset_geo25 = copy(Dataset)

        Dict_Geo25 = { geo25.upper():vector  for geo25,vector in zip(list(tokenizer_df['sensor']),np.array(embeddding_matrix))}
        Dataset["0"] = Dataset["0"].apply(lambda x:[ Dict_Geo25[geo25] for geo25 in x])
        Dataset["0"] = Dataset["0"].apply(lambda x : np.array(sum(x)/len(x)))

        dist_matrix_coss_traj = np.zeros((Dataset["0"].shape[0],Dataset["0"].shape[0]))

        for i in range(Dataset["0"].shape[0]):
            for j in range(Dataset["0"].shape[0]):
                dist_matrix_coss_traj[i][j] = nltk.cluster.cosine_distance(Dataset["0"][i],Dataset["0"][j])



        Cossine_Matrix = pd.DataFrame(dist_matrix_coss_traj)



        query_index = np.arange(0,Cossine_Matrix.shape[0])

        for i in range(0,Cossine_Matrix.shape[0]):

            Cossine_Matrix.iloc[i,i] = np.nan 


        dist_matrix_coss_sensor = np.zeros((np.array(embeddding_matrix).shape[0],np.array(embeddding_matrix).shape[0]))

        for i in range(np.array(embeddding_matrix).shape[0]):
            for j in range(np.array(embeddding_matrix).shape[0]):
                dist_matrix_coss_sensor[i][j] = nltk.cluster.cosine_distance(np.array(embeddding_matrix)[i],np.array(embeddding_matrix)[j])

        Cossine_Matrix_sensors = pd.DataFrame(dist_matrix_coss_sensor)
        CM_sensors = copy(Cossine_Matrix_sensors)
        query_index = np.arange(0,Cossine_Matrix.shape[0])

        for i in range(0,Cossine_Matrix_sensors.shape[0]):

            Cossine_Matrix_sensors.iloc[i,i] = np.nan 

        Cossine_Matrix_sensors.index = [i+ 1 for i in Cossine_Matrix_sensors.index]
        Cossine_Matrix_sensors.columns = [str(i + 1) for i in Cossine_Matrix_sensors.columns]



        columns_sensors = np.intersect1d(Eucli_Matrix_sensors.columns.values,np.intersect1d(Cossine_Matrix_sensors.columns.values,
                                                                                                Road_Matrix_sensors.columns.values))
        index_sensors = np.intersect1d(Eucli_Matrix_sensors.index.values,np.intersect1d(Cossine_Matrix_sensors.index.values,
                                                                                            Road_Matrix_sensors.index.values))
        Cossine_Matrix_sensors = Cossine_Matrix_sensors.loc[index_sensors,columns_sensors]
        Eucli_Matrix_sensors = Eucli_Matrix_sensors.loc[index_sensors,columns_sensors]
        Road_Matrix_sensors = Road_Matrix_sensors.loc[index_sensors,columns_sensors]
    
    
        LatLong = pd.read_csv("Trajectories/"+Emb_dict[list(Emb_df.Emb)[0]][0] + ".csv", index_col=0)
        LatLong["trajectory_id"] = LatLong.index
           
        Objects_loc = list(range(len(Cossine_Matrix_sensors)))
        obj_loc = Objects_loc[0]
        
    Traj_Number_Traj_id = pd.read_csv("Traj_Datasets/"+"trajetorias_roubados_representativos_18_flatten_ids_teste.csv", index_col=0)
    traj_id_dict = dict(Traj_Number_Traj_id["0"])
    traj_id_dict = { traj_id_dict[key]:key for key in list(traj_id_dict.keys())}
    
    
    LL = pd.read_csv("Sensors_Lat_Lon/"+"sensors_roubados_representativos_countmin18(1).csv", index_col='geos25')
    LL = LL.astype({'lat':'float32', 'lon':'float32'})
    LL.drop_duplicates(inplace=True)
    LL['sensor'] = [str(geos25).lower() for geos25 in LL.index.values]
    LL.set_index('sensor', inplace=True)
    
    
    dist = "Edit Distance"
    embedding = "BERT" 
    
    Objects_traj = sorted(list(Traj_Number_Traj_id.index))
    
    
    obj_type = ""
    obj_traj = Objects_traj[0]
    
    dist_topk_traj = ""
    dist_topk_loc = ""
    topk_loc = -1
    topk_traj = -1
    topk = -1
    
    out_mrr_traj = setup_ui(widgets.Label(""))
    out_mrr_traj.layout = widgets.Layout(width = "450px",border='solid 2.0px',margin='0px 5px 5px 5px',padding='2px 2px 2px 2px')
    
    out_mrr_loc = setup_ui(widgets.Label(""))
    out_mrr_loc.layout = widgets.Layout(width = "450px",border='solid 2.0px',margin='0px 5px 5px 5px',padding='2px 2px 2px 2px')
    
    out_topk_traj = setup_ui(widgets.Label(""))
    out_topk_traj.layout = widgets.Layout(width = "825px",border='solid 2.0px',margin='0px 10px 10px 10px',padding='5px 5px 5px 5px')
    
    out_topk_loc = setup_ui(widgets.Label(""))
    out_topk_loc.layout = widgets.Layout(width = "825px",border='solid 2.0px',margin='0px 10px 10px 10px',padding='5px 5px 5px 5px')
    
    
    
    def rank(dist_results, dist_correct, query_index):
        return np.sum(dist_results[query_index,:] <= dist_results[query_index, np.nanargmin(dist_correct, axis=1)[query_index]].reshape(query_index.shape[0],1),axis=1)

    def reciprocal_rank(dist_results, dist_correct, query_index):
        return 1./np.sum(dist_results[query_index,:] <= dist_results[query_index, np.nanargmin(dist_correct, axis=1)[query_index]].reshape(query_index.shape[0],1),axis=1)

    def mean_reciprocal_rank(dist_results, dist_correct, query_index):
        return np.mean(reciprocal_rank(dist_results, dist_correct, query_index))
    

    def reciprocal_rank_smallest(df_results, df_corrects, k):
        df_corrects_small =  df_corrects.nsmallest(k+1, df_corrects.columns)
        sml_idx = df_corrects_small.index.values
        df_results_small = df_results.loc[sml_idx,1:]
        return reciprocal_rank(df_results_small.values, df_corrects_small.values, np.arange(0,len(df_corrects_small)))


    def mean_reciprocal_rank_smallest(df_results, df_corrects, k):
        return np.mean(reciprocal_rank_smallest(df_results, df_corrects, k+1))

    def reciprocal_rank_filter(df_results, df_corrects, filter_correct, max_cor, max_res):
        df_corrects_small =  df_corrects[df_corrects <= filter_correct]
        df_results_small = df_results[df_corrects <= filter_correct]
        df_results_small[~df_corrects_small.notna()] = max_res
        df_corrects_small[~df_corrects_small.notna()] = max_cor
        return reciprocal_rank_filter_(df_results_small.values, df_corrects_small.values, np.arange(0,len(df_corrects_small)), max_res)

    def reciprocal_rank_filter_(dist_results, dist_correct, query_index, max_res):
        rr = np.sum(np.multiply(
          dist_results[query_index,:] <= dist_results[query_index, np.nanargmin(dist_correct, axis=1)[query_index]].reshape(query_index.shape[0],1),
          (dist_results[query_index,:] < max_res)),axis=1) 
        
        idx = rr==0
        rr = 1./rr
       
        rr[idx] = 0
        return rr
    
    def mean_reciprocal_rank_filter(df_results, df_corrects, filter_correct, max_cor, max_res):
        return np.mean(reciprocal_rank_filter(df_results, df_corrects, filter_correct, max_cor, max_res))

    
    def MRR_Plot_Loc(b):
        
        with out_mrr_loc:
            
            out_mrr_loc.layout = widgets.Layout(width = "450px",border='solid 2.0px',margin='0px 5px 5px 5px',padding='2px 2px 2px 2px')
            out_mrr_loc.clear_output()
            
            global Cossine_Matrix_sensors
            global Eucli_Matrix_sensors 
            global Road_Matrix_sensors
            
            
            mrre=[]
            for ed in range(500,10001,500):
                mrr_ = mean_reciprocal_rank_filter(Cossine_Matrix_sensors.loc[Eucli_Matrix_sensors.index.values,:], Eucli_Matrix_sensors, ed,ed+1,2)
                mrre.append(mrr_)


            mrr=[]
            for rd in range(500,10001,500):
                mrr_ = mean_reciprocal_rank_filter(Cossine_Matrix_sensors, Road_Matrix_sensors, rd,rd+1,2)
                mrr.append(mrr_)

            plt.plot( range(500,10001,500), mrr, '-*')

            plt.plot( range(500,10001,500), mrre, '-o')

            plt.legend(['Road Distance', 'Euclidean Euclidean'])
            plt.xlabel('distance (meters)')
            plt.ylabel('mrr')
            plt.title('Mean Reciprocal Ranking vs Range Distance')
            plt.savefig('mrr_correct',format='pdf')
            plt.show()
            
            out_mrr_loc.layout = widgets.Layout(width = "450px",border='solid 2.0px white',margin='0px 5px 5px 5px',padding='2px 2px 2px 2px')
            
    
    def MRR_Plot_Traj(b):
        
        global Cossine_Matrix
        global Edit_Matrix
        global DTW_Matrix 
        
    
        with out_mrr_traj:
            
            out_mrr_traj.layout = widgets.Layout(width = "450px",border='solid 2.0px',margin='0px 5px 5px 5px',padding='2px 2px 2px 2px')
            out_mrr_traj.clear_output()


            mrrdtw=[]
            for dtw in range(5,100,5): 
                mrr_ = mean_reciprocal_rank_filter(Cossine_Matrix.loc[DTW_Matrix.index.values,:], DTW_Matrix , dtw, dtw+1,2)
                mrrdtw.append(mrr_)
                
            

            mrredit=[]
            for d in range(5,100,5): 
                mrr_ = mean_reciprocal_rank_filter(Cossine_Matrix, Edit_Matrix, d, d+1,2)
                mrredit.append(mrr_)
                
            

            plt.plot( range(5,100,5), mrrdtw, '-*')

            plt.plot( range(5,100,5), mrredit, '-o')

            plt.legend(['DTW', 'Edit distance'])
            plt.xlabel('distance')
            plt.ylabel('mrr')
            plt.title('Mean Reciprocal Ranking vs Maximal Distance')

            plt.show()

            out_mrr_traj.layout = widgets.Layout(width = "450px",border='solid 2.0px white',margin='0px 5px 5px 5px',padding='2px 2px 2px 2px')
    
            
    def Topk_Traj_Plot(b):
        
        
        with out_topk_traj:
            
            out_topk_traj.clear_output()
            
            global Dataset
            global dist_topk_traj
            global obj_traj
            global topk_traj
            global LatLong
            global Traj_Number_Traj_id
            global traj_id_dict
            global Cossine_Matrix
           
            traj_matrix_index = Dataset[Dataset["trajectory_number"] == str(obj_traj)].index[0]
            k = int(topk_traj)
            
            cossine_matrix = Cossine_Matrix.to_numpy() 
            
            neighborhood_topk = []

            for i in range(cossine_matrix.shape[0]):
                neighborhood_topk.append((cossine_matrix[i][traj_matrix_index], i))

            neighborhood_topk_index = sorted(neighborhood_topk)
            

            neighborhood_topk_index = neighborhood_topk_index[0:k+1]
            

            trajectories = [Dataset["trajectory_number"].loc[nb[1]] for nb in neighborhood_topk_index]

            
            trajectories_id = list(Traj_Number_Traj_id["0"].loc[[int(tj) for tj in trajectories]])

            trajectories_df = pd.concat([filters.by_label(LatLong, value = tj, label_name = "trajectory_id") for tj in trajectories_id],axis=0)
            
            trajectories_df['trajectory_id'] = trajectories_df['trajectory_id'].apply(lambda x : "Trajectory : "+str(traj_id_dict[x]))
            
            move_df = MoveDataFrame(data= trajectories_df, latitude="lat", longitude="lon", datetime="time", traj_id='trajectory_id')
            #figure = folium.Figure(width=800, height=300) 
            display(f.plot_trajectories(move_df))
            
            
    
    def Topk_Loc_Plot(b):
        
        with out_topk_loc:
            
            out_topk_loc.clear_output()
            
            global CM_sensors
            global tokenizer_df 
            global LL
            global obj_loc
            global topk_loc
            
            neighborhood_topk_s = []
            cossine_matrix_s = CM_sensors.to_numpy() 

            for i in range(cossine_matrix_s.shape[0]):
                neighborhood_topk_s.append((cossine_matrix_s[i][int(obj_loc)], i))
            
            neighborhood_topk_s_index = sorted(neighborhood_topk_s)
            neighborhood_topk_s_index = [(n[0],n[1]+1) for n in neighborhood_topk_s_index]

            
            neighborhood_topk_s_index = neighborhood_topk_s_index[0:int(topk_loc)+1]
           

            closest_labels = tokenizer_df.loc[[n[1] for n in neighborhood_topk_s_index], :]
            nb = [n[1]-1 for n in neighborhood_topk_s_index]
            closest_sensor = LL.loc[closest_labels['sensor'].apply(lambda x : x.lower())]
            closest_sensor['id'] = ["Location: "+str(n) for n in nb]
            closest_sensor['datetime'] = 0.0
            
            
            map = plot_trajectories(closest_sensor)
            #figure = folium.Figure(width=800, height=300) 
            display(plot_points(closest_sensor, user_point='gray', base_map=map))
        
    
    def on_change_traj_data(change):

        global traj_data
        global Dataset
        global Objects_loc
        global LatLong
        global Cossine_Matrix
        global DTW_Matrix
        global Edit_Matrix
        global Cossine_Matrix_sensors
        global Eucli_Matrix_sensors 
        global Road_Matrix_sensors
        global tokenizer_df
        global embeddding_matrix
        global obj_loc
        
        traj_data = change.new
        
            
        Dataset = pd.read_csv("Traj_Datasets/"+ Labels["External Sensors Trajectories"])
    
        Dataset["0"] = Dataset["0"].apply(lambda x : x.replace("[","").replace("]","").replace(",","").replace("\n","").replace("'",""))
        Dataset["0"] = Dataset["0"].apply(lambda x : x.split())
        Dataset.rename(columns = {'Unnamed: 0':'trajectory_number'}, inplace = True)
        Dataset["trajectory_number"] = Dataset["trajectory_number"].apply(lambda x: str(x))
        
        Dataset_geo25 = copy(Dataset)

        Dict_Geo25 = { geo25.upper():vector  for geo25,vector in zip(list(tokenizer_df['sensor']),np.array(embeddding_matrix))}
        Dataset["0"] = Dataset["0"].apply(lambda x:[ Dict_Geo25[geo25] for geo25 in x])
        Dataset["0"] = Dataset["0"].apply(lambda x : np.array(sum(x)/len(x)))

        dist_matrix_coss_traj = np.zeros((Dataset["0"].shape[0],Dataset["0"].shape[0]))

        for i in range(Dataset["0"].shape[0]):
            for j in range(Dataset["0"].shape[0]):
                dist_matrix_coss_traj[i][j] = nltk.cluster.cosine_distance(Dataset["0"][i],Dataset["0"][j])

        Cossine_Matrix = pd.DataFrame(dist_matrix_coss_traj)
        CM_sensors = copy(Cossine_Matrix_sensors)

        query_index = np.arange(0,Cossine_Matrix.shape[0])

        for i in range(0,Cossine_Matrix.shape[0]):

            Cossine_Matrix.iloc[i,i] = np.nan 


        dist_matrix_coss_sensor = np.zeros((np.array(embeddding_matrix).shape[0],np.array(embeddding_matrix).shape[0]))

        for i in range(np.array(embeddding_matrix).shape[0]):
            for j in range(np.array(embeddding_matrix).shape[0]):
                dist_matrix_coss_sensor[i][j] = nltk.cluster.cosine_distance(np.array(embeddding_matrix)[i],np.array(embeddding_matrix)[j])

        Cossine_Matrix_sensors = pd.DataFrame(dist_matrix_coss_sensor)

        for i in range(0,Cossine_Matrix_sensors.shape[0]):

            Cossine_Matrix_sensors.iloc[i,i] = np.nan 

        Cossine_Matrix_sensors.index = [i+ 1 for i in Cossine_Matrix_sensors.index]
        Cossine_Matrix_sensors.columns = [str(i + 1) for i in Cossine_Matrix_sensors.columns]
        
        Eucli_Matrix_sensors.columns = [str(i) for i in Eucli_Matrix_sensors.columns]
        Road_Matrix_sensors.columns = [str(i) for i in Road_Matrix_sensors.columns]
        
        columns_sensors = np.intersect1d(Eucli_Matrix_sensors.columns.values,np.intersect1d(Cossine_Matrix_sensors.columns.values,
                                                                                            Road_Matrix_sensors.columns.values))
        index_sensors = np.intersect1d(Eucli_Matrix_sensors.index.values,np.intersect1d(Cossine_Matrix_sensors.index.values,
                                                                                        Road_Matrix_sensors.index.values))
        Cossine_Matrix_sensors = Cossine_Matrix_sensors.loc[index_sensors,columns_sensors]
        Eucli_Matrix_sensors = Eucli_Matrix_sensors.loc[index_sensors,columns_sensors]
        Road_Matrix_sensors = Road_Matrix_sensors.loc[index_sensors,columns_sensors]

        
        Objects_loc = list(range(len(Cossine_Matrix_sensors)))
        
    
        aux1 = []
        for i in range(len(Dataset_geo25["0"])):
            aux1 += Dataset_geo25["0"][i]
    
    
        Objects_loc = sorted(list(set(aux1)))
        obj_loc = Objects_loc[0]
            
        

    def on_change_dist(change):

        global obj_type
        global Dataset
        dist = change.new
        
    def on_change_object_traj(change):
        
        global obj_traj
        obj_traj = change.new
                        
    
    def on_change_object_loc(change):
        
        global obj_loc
        obj_loc = change.new
    
    def on_change_topk_loc(change):
        
        global topk_loc
        topk_loc = change.new
        
    def on_change_topk_traj(change):
        
        global topk_traj
        topk_traj = change.new
        
    def on_change_embedding_choice(change):
        
        global embedding
        global Cossine_Matrix
        global Edit_Matrix
        global DTW_Matrix
        global Cossine_Matrix_sensors
        global Eucli_Matrix_sensors
        global Road_Matrix_sensors
        global query_index
        global Emb_dict
        global Emb_list
        global tokenizer_df
        global embeddding_matrix
        global text_dataset_out
        
        embedding = change.new
        
        with text_dataset_out:
            
            text_dataset_out.clear_output()
        
            text_dataset.options = [''] if(not(embedding in Emb_dict.keys())) else Emb_dict[embedding] + ['']
            
            text_dataset.value = ''
            
            display(text_dataset)
            
        aux_emb = pd.read_csv("Embeddings/" + embedding + ".csv") 
    
        tokenizer_df = aux_emb.loc[0:list(pd.isnull(aux_emb["sensor"])).index(True)-1,["sensor","id"]]
        tokenizer_df.id = [int(i) for i in tokenizer_df.id]
        tokenizer_df.index = tokenizer_df.id
        tokenizer_df = tokenizer_df[["sensor"]]

        embeddding_matrix = aux_emb.loc[list(pd.isnull(aux_emb["sensor"])).index(True):int(list(pd.isnull(aux_emb["sensor"])).index(True)*2),
                        [str(i) for i in range(len(aux_emb.columns)-2)]]
        
        embeddding_matrix.index = [i for i in range(embeddding_matrix.shape[0])]
    
    
    
    
    
    text_dataset = widgets.Select(description="Select Data:",layout=widgets.Layout(width="380px",height="180px"),
                           options= [''] if(not(Emb_list[0] in Emb_dict.keys())) else Emb_dict[Emb_list[0]] + [''],value='')
    text_dataset.observe(on_change_traj_data,names="value")
    text_dataset_out = setup_ui(text_dataset)
    
    model_text = widgets.Label("Select Embedding")
    model_choice = widgets.Dropdown(description='',layout=widgets.Layout(width="150px"),options=Emb_list,value=Emb_list[0])
    model_choice.observe(on_change_embedding_choice,names="value")
    model_choice_out = setup_ui(model_choice)
    
    
    
    


    l5 = widgets.Label("Select Trajectory")
    text18 = widgets.Dropdown(description='',options=Objects_traj,layout=widgets.Layout(width="150px"))
    text18.observe(on_change_object_traj,names="value")
    text18_output = setup_ui(text18)
    
    l5_2 = widgets.Label("Select Loacation")
    text18_2 = widgets.Dropdown(description='',options=Objects_loc,layout=widgets.Layout(width="150px"))
    text18_2.observe(on_change_object_loc,names="value")
    text18_output_2 = setup_ui(text18_2)

    text_k = widgets.Label("Top-k")
    text19 = widgets.Text(description='',layout=widgets.Layout(width="60px"))
    text19.observe(on_change_topk_traj,names="value")
    
    text_k_2 = widgets.Label("Top-k")
    text19_2 = widgets.Text(description='',layout=widgets.Layout(width="60px"))
    text19_2.observe(on_change_topk_loc,names="value")


    b6 = widgets.Button(description="Plot", layout=widgets.Layout(width="60px"))
    b6.on_click(Topk_Traj_Plot)
    b6.style.button_color = "lightgray"
    
    b6_2 = widgets.Button(description="Plot", layout=widgets.Layout(width="60px"))
    b6_2.on_click(Topk_Loc_Plot)
    b6_2.style.button_color = "lightgray"
    
    b7 = widgets.Button(description="Plot", layout=widgets.Layout(width="60px"))
    b7.on_click(MRR_Plot_Traj)
    b7.style.button_color = "lightgray"
    
    b7_2 = widgets.Button(description="Plot", layout=widgets.Layout(width="60px"))
    b7_2.on_click(MRR_Plot_Loc)
    b7_2.style.button_color = "lightgray"
    
    acc_mrr_traj = widgets.VBox([spc, widgets.HBox([spc,spc,spc,widgets.HBox([widgets.Label("MRR"),spc,b7])]),widgets.VBox([spc,spc,out_mrr_traj])])
    
    acc_mrr_loc = widgets.VBox([spc, widgets.HBox([spc,spc,spc,widgets.HBox([widgets.Label("MRR"),spc,b7_2])]),widgets.VBox([spc,spc,out_mrr_loc])])
    
    acc_similar_traj = widgets.HBox([widgets.VBox([widgets.HBox([widgets.HBox([l5,spc, text18_output]),spc, widgets.HBox([text_k,spc,text19,spc,spc,b6])]),
          widgets.HBox([spc,spc,spc,widgets.VBox([spc,out_topk_traj])])])])
    
    acc_similar_loc = widgets.VBox([widgets.VBox([widgets.HBox([widgets.HBox([l5_2,spc, text18_output_2]),spc, widgets.HBox([text_k_2,spc,text19_2,spc,spc,b6_2])]),
          widgets.HBox([spc,spc,spc,widgets.VBox([spc,out_topk_loc])])])])
    
    accordion1 = widgets.Accordion(children=[acc_mrr_loc],selected_index=None)
    accordion2 = widgets.Accordion(children=[acc_similar_loc],selected_index=None)
    accordion3 = widgets.Accordion(children=[acc_mrr_traj],selected_index=None)
    accordion4 = widgets.Accordion(children=[acc_similar_traj],selected_index=None)
    
    accordion1.set_title(0, 'Mean Reciprocal Rank (MRR) Location')
    accordion2.set_title(0, 'Similar Locations')
    accordion3.set_title(0, 'Mean Reciprocal Rank (MRR) Trajectory')
    accordion4.set_title(0, 'Similar Trajectories')
    

    h15 = widgets.VBox([widgets.HBox([widgets.VBox([model_text,model_choice_out]),spc,spc,spc1, widgets.VBox([text_dataset_out])]),spc, accordion1,spc,accordion2,spc,accordion3,spc,accordion4])
    
    display(h15)
  

# Trajectory Data / Trajectory Embedding

# TrajEmb Upload: Embedding Importado / 
# Visualize: Em vez janela de texto um dataframe; Calculate = Show Statistics; View Trajectory Data

# Extrinsic Eval: Trajectory Dataset (select); Embedding Model; Dataframe com as estatisticas : Comparativo entre Embediing 
# Similar Object/ MRR Plot
# Location vs Trajectory, Edit e DTW

# 1º aba: Intrinsic Evaluation - Matheus 
# 2º aba: Extrinsic Evaluation - Wilken

# colocar uma label ao subir o arquivo
# Botar código no github

#locations Primeiro
# Euclidiana e a distância na Rede (MRR)
# cosseno entre as trajetória
# model -> Embedding 

# 1 Location MRR
# 2 Location Similar
# 3 Trajectories MRR
# 4 Trajectories Similar

# O Embedding vai influenciar a matriz cosseno pra comparar com as outras matrizes de distância e na matriz cosseno pra calcular
# os elementos e as trajetórias mais próximas
# Colocar em vez de Id de placa o numero
#Colocar o Id do sensor
# MRR -> Mean Recip Rank

# Onde tem LBERT - > BERT Embedding + LSTM Model
# Onde tem LBERT-FT -> BERT Embedding + LSTM Model + Fine Tuning
# Onde tem LW2v -> LSTM + CBOW Embedding
# Onde tem LW2V-FT - > LSTM + CBOW Embedding + Fine Tuning
# Df_Roubados -> Trajectories

#Além de modularizar em funções, padronizar com PEP 8, inclusive as variáveis, criar o script .sh para o voilá
#temos que discutir o que vai ser gerado pela ferramenta e o que já vai está nela
#porque os dados, inclusives as matrizes e tokenizadores vão modeificar a medida que novos dados vão chegando e alimentando aferramenta
#ai nesse caso faz um update e salva nas pastas?
#e a ferramente vai ser capaz de treinar os modelos?
#colocar elemnetos de coisa carregando como o tqdm
#tem que colocar a parte referente ao CBOW pra já ter uma lógica
#modularizar em arquivos .py
#comentar e se possível documentar
#distribuir o conhecimento da ferramenta pra não ficar só com uma pessoa sabendo um bocado sobre a ferramenta
#melhorar as explicações la no github
#PEP 9 tb
#usar tecnicas de padrão de projeto como factory
#dsicutir as pastas e o uso de novos modelos em novos dados, treinamento
#dsicutir a respeito do tratamento e validação de um tipo específico de dataset
#e o tratamento que é dado a ele
#cálculo dos grid de trajetória e a melhor forma de plotar as trajetóricas e os locations
#melhor biblioteca para a plotagem dos gráficos, gráficos maiores?
#...
#vão surgir novas demandas
#...
#usar os try except, para ficar fácil a depuração
#...

# De onde vem os .h5?

# Upload Embedding => Tokenizer + Matriz de Embedding

# A Ferramenta de Gerar os modelos, os Embeddings, a partir dos datasets

# pre-processamento dos textos

# try-except // Label amigavel

# Possibilidade de Outras distâncias e calculo das outras matrizes de distância para outros datasets.
# Intrinsic Evaluation calcular as matrizes para location e para trajetorias.
# User_Labe =>Datset_Label
# Uma Aba dentro do acoodion, multiselect

# Remove and Replace na propria tab de upload
# caixa de texto Rename; clicar e aparecer o nome la embaixo

# ajeitar o layaout pra parar de cortar

# Selec Embedding